In [1]:
from py2neo import cypher, Graph, NodeMatcher, RelationshipMatcher, Subgraph, Walkable, Path, Node
import pandas as pd
import igraph
import numpy as np
from dfply import *
from IPython.display import display
from sqlalchemy import create_engine, event
from sqlalchemy.engine.url import URL
import datetime
import time
from treelib import Node, Tree
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Trusted Connection to Named Instance
conn_str = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=KVHSQLPC56;DATABASE=AHDA;Trusted_Connection=yes;'
# Create a URL for the sqlalchemy engine to use
conn_url = URL.create("mssql+pyodbc", query={"odbc_connect": conn_str})
# Create the connection
engine = create_engine(conn_url)


class sql:
    
    def __init__(self,cohort_id=0):
        
        # Trusted Connection to Named Instance
        conn_url = URL.create("mssql+pyodbc", query={"odbc_connect": conn_str})
        # Create the connection
        self.engine = create_engine(conn_url)
        self.cohort_id = cohort_id
        self.interactions = pd.DataFrame()


    def patients_projection2(self):
        
        svc=self.interactions.service_class_id
        svc_unique = np.unique(svc)
        map={}
        for svc in svc_unique:
            index=np.where(self.interactions.service_class_id==svc)[0]
            patients=self.interactions.iloc[index].patient_id

            patients_unique=np.unique(patients)
    
            for p1 in patients_unique:
                for p2 in patients_unique:
                    if p1<p2:
                        if p1 in map:
                            result=map.get(p1)
                            if p2 in result:
                                weight=result.get(p2)
                                result[p2]=weight+1
                            else:
                                result[p2]=1

                        else:
                            result={}
                            result[p2]=1
                        map[p1]=result
                        
                        
                        

        df3 = pd.DataFrame()
        source=[]
        target=[]
        weight=[]

        for key in map:
            neighbor=map.get(key);
            for n in neighbor:
               
                source.append(key)
                target.append(n)
                weight.append(neighbor.get(n))
                
                
        df3= pd.DataFrame({'source': source, 'target': target,'weight':weight})
                

        #df3.insert(loc=0,
                         #column='source',
                             # value=source)
       # df3.insert(loc=1,
                        # column='target',
                             # value=target)

       # df3.insert(loc=2,
                        # column='weight',
                              #Svalue=weight)
        return df3


  

    def patients_projection44(self):
        svc = self.interactions.service_class_id
        svc_unique = np.unique(svc)
        patient_map = defaultdict(lambda: defaultdict(int))

        for svc in svc_unique:
            index = np.where(self.interactions.service_class_id == svc)[0]
            patients = self.interactions.iloc[index].patient_id
            patients_unique = np.unique(patients)

            for p1, p2 in combinations(patients_unique, 2):
                patient_map[p1][p2] += 1

        source = []
        target = []
        weight = []

        for p1, neighbors in patient_map.items():
            for p2, count in neighbors.items():
                source.append(p1)
                target.append(p2)
                weight.append(count)

        df3 = pd.DataFrame({'source': source, 'target': target, 'weight': weight})
        return df3
    
    
    def patients_interactions(self):
      
        if self.cohort_id != -1:
            query = """SELECT i.patient_id,service_class_id,start_datetime  FROM  HF.interactions i
                 INNER JOIN HF.patient_cohort_list pcl on pcl.patient_id = i.patient_id 
                 WHERE pcl.cohort_id ="""+str(self.cohort_id)
        
        if self.patient_id_query  != "":
            query =  self.patient_id_query 
            
        if len(self.patient_id_df)  > 0:
            # patient_list = self.patient_id_df["patient_id"].tolist()
            patient_string = '\',\''.join(self.patient_id_df["patient_id"])
            patient_string = '\'' + patient_string + '\''
            query = """SELECT i.patient_id,service_class_id,start_datetime  FROM  HF.interactions i
                 INNER JOIN HF.demographics d on d.patient_id = i.patient_id 
                 WHERE d.patient_id IN (""" + patient_string + """) """
            """WITH FirstFall AS (""" + patient_string + """)
                SELECT 
                       Distinct i.patient_id, i.service_class_id, i.start_datetime, i.end_datetime
                FROM 
                       FirstFall
                INNER JOIN 
                       HF.interactions i ON i.patient_id = FirstFall.patient_id
                WHERE 
                       i.start_datetime < first_fall

                AND 
                        i.service_class_id NOT IN (80,150,152,68)

                ORDER BY i.patient_id, i.start_datetime"""
            
       # query += """ ORDER BY i.patient_id,start_datetime"""


        df = pd.read_sql(query,engine)
       
        #interactions= pd.read_sql(query,engine)
        # result=df.merge(patient_id_df, how='inner', on = 'patient_id')
        # self.interactions=result[result['patient_id'].notna()]
        # self.interactions=df
        # return self.interactions
        return df
        
                    
    def sequence(self):
      
        
       # query = '''SELECT service_class_id ,patient_id,start_datetime FROM  HF.interactions
       #WHERE patient_id IN (SELECT patient_id FROM HF.patient_cohort_list WHERE cohort_id ='''+''+str(self.cohort_id)+") ORDER BY start_datetime, patient_id "
        #  df=
         #df = pd.read_sql(query,self.engine)
        df=self.interactions
        df = df.astype({"service_class_id": str}, errors='raise')
        #df4=df.groupby(by='patient_id',as_index = False).agg({
                                     # 'seq':lambda x: list(x)})
            
        df4=df.groupby(by='patient_id',as_index = False).agg({'service_class_id': ' '.join})
                                     #'seq':lambda x: list(x)})    
        # df4=pd.read_csv("H:/Projects/MSF_Covid/ErnieShare/DATA/coh04_schiz_pats.csv")
        return df4                    
            
            
    def tf_idf_matrix(self):
 
        corpus=self.sequence()
       
            
       
        
        corpus.columns = ['pat','svc']

        self.svcCodes = corpus[['svc']]
        patHD = corpus[['pat']]
        corpus_df = self.svcCodes[0:]
        corpus_df = corpus_df.squeeze()
        cv = CountVectorizer(token_pattern='\\b(\\w+)\\b')
        cv_matrix = cv.fit_transform(corpus_df)
        cv_matrix = cv_matrix.toarray()


        # get all unique words in the corpus
        vocab = cv.get_feature_names_out()
        # show document feature vectors
        df = pd.DataFrame(cv_matrix, columns=vocab)
        self.svcINT = vocab.astype(int)
        a=df.columns
        self.service_class_id = pd.DataFrame(a.transpose().to_list(),columns=['service_class_id'])
        dfpat = pd.concat([patHD,df],axis=1)
        dfpat.iloc[[0]]
        dfT = dfpat.transpose()
        xyz = dfT.iloc[[0]].values[0].tolist()
        dfT.columns = xyz
        dfU = dfT[1:]
        tv = TfidfTransformer()
        

        tv_matrix = tv.fit_transform(dfU)
        tv_matrix = tv_matrix.toarray()
        tvocab = tv.get_feature_names_out()
     
        rawTFIDF = pd.DataFrame(np.round(tv_matrix, 2), columns=patHD)
        rawTFIDF = pd.concat([self.service_class_id,rawTFIDF],axis=1)
        abcd = pd.DataFrame(np.round(tv_matrix, 2), columns=tvocab)
        TF = rawTFIDF.copy(deep=True)

        self.similarity_matrix = cosine_similarity(tv_matrix)
        similarity_df = pd.DataFrame(self.similarity_matrix)
        similarity_df.loc['Total'] = round(similarity_df.sum()/len(similarity_df)*100,2)
        simCol = similarity_df.iloc[[-1][:]].transpose()
        self.simCol = simCol.rename(columns={'Total':'Sim.%'})
        return rawTFIDF


    def numberofpatients(self,interaction,node):
       
        index=np.where(interaction.service_class_id==node)[0]
        df=interaction.iloc[index]
        
        patients=df.patient_id.unique()                     
        numberofP=len(patients)
        return numberofP
		
		
		
    def normalized_weight_service_projection(self):
        df=self.interactions
       
        result1=self.service_class_projection() 
        
        
        
        df=df[['service_class_id','patient_id']]
        df=df.drop_duplicates()
        df2 = df.groupby(['service_class_id'],as_index=False).count()
        df2=df2.rename(columns={'service_class_id': 'service_class_id_x','patient_id': 'coun1t'})
        df3=df2.rename(columns={'service_class_id_x': 'service_class_id_y','patient_id': 'count2'})
        result1=result1.rename(columns={'count': 'weight'})
        x_count=pd.merge(result1, df2, how="left", on='service_class_id_x')
        x_y_count=pd.merge(x_count, df3, how="left", on='service_class_id_y')
        y=x_y_count
        y['normalized_weight']=y['weight']/((y['coun1t_x']+y['coun1t_y'])-y['weight'])
        #y=y[['service_class_id_x','service_class_id_y','normalized_weight']]
        display(y)
        return y
                                                                     


In [2]:
class communityDetection:
    
    def __init__(self,cohort_id=-1, patient_id_df='',patient_id_query='',interactions=''):
            '''self.patient_id_df=patient_id_df
            self.cohort_id=cohort_id
            self.patient_id_query=patient_id_query
            self.sql=sql(cohort_id=self.cohort_id)
            self.sql.patient_id_df=patient_id_df
            self.sql.cohort_id=cohort_id
            self.sql.patient_id_query=patient_id_query
            
            self.sql.interactions = self.sql.patients_interactions()'''
            self.sql=sql()
            self.sql.interactions=interactions
           
            
    def takeFromNeo(self):
        g =Neo4j('bolt://localhost:7687', 'neo4j', 'New_Neo4j')
        df=pd.read_csv('cohort52new.csv')
        self.patients=g.patients()
        display(self.patients)
        #df=graph.query(b=self.cohort_id)
        edges, weights = [], []
        # Iterate over each row in the csv using reader object
        for i in range(len(df)):
            row=df.iloc[i]
            u, v, weight = row[0],row[1],row[2]
            
            edges.append((u,v))
            weights.append(float(weight))
        self.graph = igraph.Graph(edges, edge_attrs={"weight": weights})
         #self.label=graph.label()
        
       # self.label=self.label.iloc[0:253]
         #self.label= pd.read_csv('labelnew2.csv')
         #return self.patients
        #display(self.label)
        #creating adjacency list of the graph

    def create_graph_for_patients2(self):
               
        df=self.sql.patients_projection2()
        print("start")
        

        #df=graph.query(b=self.cohort_id)
        edges, weights = [], []
        # Iterate over each row in the csv using reader object
        self.graph2= igraph.Graph.TupleList(df.itertuples(index=False), directed=False, weights=False, edge_attrs="weight")


    def create_graph_for_patients(self):
        
        
        #self.patients=self.sql.patients()
        
        
       # df=self.sql.patinet_projection()
        df=self.sql.patients_projection2()
        #df=self.sql.patients_projection2()
        
        map1={}   
        map2={}
        l1=np.unique(df.target)
        l2=np.unique(df.source)
        list1 = l1.tolist()
        list2 = l2.tolist()
        list3=list1+list2
        arr = np.array(list3)
        unique_patients=np.unique(arr)
        len(unique_patients)

        #for mappping
        i=0
        while i <len(unique_patients):
            vertex=unique_patients[i]
            map1[vertex]=i
            map2[i]=vertex
            i=i+1

        #transfer a df of strings to dataframe of integers with using map
        source=[]
        target=[]
        weight=[]
        for i in range(len(df)):
            row=df.iloc[i]
            u, v, weight2 = row[0],row[1],row[2]

            source.append(map1.get(u))
            target.append(map1.get(v))
            weight.append(weight2)

        df2= pd.DataFrame()

        df2.insert(loc=0,
                     column='source',
                 value=source)

        df2.insert(loc=1,
                         column='target',
                    value=target)
        df2.insert(loc=2,
                         column='weight',
                    value=weight)




        
        
        print("done!")
        #df=graph.query(b=self.cohort_id)
        edges, weights = [], []
        # Iterate over each row in the csv using reader object
        for i in range(len(df)):
            row=df2.iloc[i]
            u, v, weight = row[0],row[1],row[2]
            
            edges.append((u,v))
            weights.append(float(weight))
        self.graph2 = igraph.Graph(edges, edge_attrs={"weight": weights})
        self.map3=map1
        self.map4=map2
        
        
    
    def community_detection_on_patinets(self, numberofiterations=-1):
        
        numberofitterations=numberofiterations
        # first itteration
        louvain = self.graph2.community_multilevel(weights=self.graph2.es['weight'], return_levels=False)
        pi = []
        piI = []

        numberofclustures = 0
        for i in range(len(louvain)):
            component = louvain[i]

            if (len(component) > 1):
                piI.append(component)

        pi.append(piI)
        itteration = 1
        if numberofitterations==-1:
            while True:
                piI1 = []
                Previous_components = pi[itteration - 1]
                for community in pi[itteration - 1]:

                    # print(community)
                    induced_subgraph = self.graph2.induced_subgraph(list(community))
                    louvain1 = induced_subgraph.community_multilevel(weights=induced_subgraph.es['weight'], return_levels=False)
                    for i in range(len(louvain1)):
                        subgraph = louvain1[i]
                        # print(subgraph)
                        lst = []
                        for maped_vertex in subgraph:
                            map = community[maped_vertex]
                            lst.append(map)
                        # print(lst)
                        # print(tuple(lst))

                        piI1.append((lst))
                pi.append(piI1)
                current_components = pi[itteration]
                if not current_components > Previous_components:
                    break
                else:
                    itteration += 1
        if numberofitterations!=-1:
            if numberofitterations == -1:
                while True:
                    piI1 = []
                    Previous_components = pi[itteration - 1]
                    for community in pi[itteration - 1]:

                        # print(community)
                        induced_subgraph = self.graph2.induced_subgraph(list(community))
                        louvain1 = induced_subgraph.community_multilevel(weights=induced_subgraph.es['weight'],
                                                                         return_levels=False)
                        for i in range(len(louvain1)):
                            subgraph = louvain1[i]
                            # print(subgraph)
                            lst = []
                            for maped_vertex in subgraph:
                                map = community[maped_vertex]
                                lst.append(map)
                            # print(lst)
                            # print(tuple(lst))

                            piI1.append((lst))
                    pi.append(piI1)
                    current_components = pi[itteration]
                    if not current_components > Previous_components:
                        break
                    else:
                        itteration += 1


            if numberofitterations != -1:
                while itteration<numberofitterations:
                    piI1 = []
                    Previous_components = pi[itteration - 1]
                    for community in pi[itteration - 1]:

                        # print(community)
                        induced_subgraph = self.graph2.induced_subgraph(list(community))
                        louvain1 = induced_subgraph.community_multilevel(weights=induced_subgraph.es['weight'],
                                                                         return_levels=False)
                        for i in range(len(louvain1)):
                            subgraph = louvain1[i]
                            # print(subgraph)
                            lst = []
                            for maped_vertex in subgraph:
                                map = community[maped_vertex]
                                lst.append(map)
                            # print(lst)
                            # print(tuple(lst))

                            piI1.append((lst))
                    pi.append(piI1)
                    current_components = pi[itteration]
                    if not current_components > Previous_components:
                        break
                    else:
                        itteration += 1


        
        print("number of all iterations",len(pi))
        for i in range(len(pi)):
            iteration=pi[i]
            print("number of commmunities in "+str(i+1)+' '+"iteration",len(iteration ))
      
    
       #prevent calling community detection for patients in create table function:
       #return pi,map1,map2
        self.pi6=pi
        return self.pi6,self.graph2
           
    
    def create_table_for_patients(self,iteration2=-1):
        itteration2=iteration2
        if itteration2==-1:
            itteration=len(self.pi6)
            print(itteration)
        if itteration2 !=-1:
            itteration=itteration2
        weight = []
        out_degree=[]
        weightd_degree=[]
        for j in range(len(self.pi6)):
            itteration1 = self.pi6[j]

            weightI = []
            out_degreeI = list()
            weightd_degreeI=[]
            for i in range(len(itteration1)):
                component = itteration1[i]

                subgraph = self.graph2.induced_subgraph(component)
                total_degree = self.graph2.strength(component, mode='all', loops=True, weights="weight")
                weightd_degreeI.append(total_degree)
                lst = list(range(len(component)))
                in_degree = subgraph.strength(lst, mode='all', loops=True, weights="weight")
                weightI.append(in_degree)
                lst2=[]
                for item1, item2 in zip(total_degree,in_degree ):
                    item = item1 - item2
                    lst2.append(item)
                out_degreeI.append(lst2) 

            weight.append(weightI)
            out_degree.append(out_degreeI)
            weightd_degree.append(weightd_degreeI)


        if itteration2==-1:
            itteration=len(self.pi6)
            #for community id
            
            self.vertices = []
            idd = 1
            id = []
            age=[]
            gender=[]
            primary=[]
            fsa=[]
            n_encounters=[]
            n_interactions=[]
            first_admit_dt=[]
            last_admit_dt=[]
            covid_positive=[]
            
            
            
            
            
            for component in self.pi6[itteration - 1]:
                for vertex in component:
                    v= self.map4.get(vertex)
                    self.vertices.append(v)
                    id.append(idd)
                idd = idd + 1
                    
                
                   
                   #f b=label.name[index]


            #for weight
            weightt=[]
            for component in weight[itteration - 1]:
                for degree in component:
                    weightt.append(degree)



            #for out degree       
            out_degree2=[]   
            for component in out_degree[itteration - 1]:
                  for out_deg in component:
                        out_degree2.append(out_deg)

           #for weighted degree       
            weighted_degree2=[]   
            for component in weightd_degree[itteration - 1]:
                  for weight_deg in component:
                        weighted_degree2.append(weight_deg)             





            df = pd.DataFrame()

            df.insert(loc=0,
                      column='patients_id',
                      value=self.vertices)

            
                   
            
            df.insert(loc=1,
                      column='community_id',
                      value=id)

            df.insert(loc=2,
                      column='in_degree',
                      value=weightt)
            df.insert(loc=3,
                        column='out_degree',
                        value=out_degree2)
            


            df.insert(loc=4,
                        column='weighted_degree',
                       value=weighted_degree2)
            
            

     
            return(df)

        if itteration2!=-1:
            #for id
            # idd = 1
            # id = []
            # for component in self.pi6[itteration - 1]:
                # for vertex in component:
                    # id.append(idd)
                # idd = idd + 1
                
            self.vertices = []
            idd = 1
            id = []
            age=[]
            gender=[]
            primary=[]
            fsa=[]
            n_encounters=[]
            n_interactions=[]
            first_admit_dt=[]
            last_admit_dt=[]
            covid_positive=[]
            
            
            
            
            
            for component in self.pi6[itteration - 1]:
                for vertex in component:
                    v= self.map4.get(vertex)
                    self.vertices.append(v)
                    '''index = np.where(self.patients.patient_id== v)[0]'''
                    id.append(idd)
                idd = idd + 1 
                  

            #for weight
            weightt=[]
            for component in weight[itteration - 1]:
                for degree in component:
                    weightt.append(degree)
            df = pd.DataFrame()                   
                    
                    

            #for out degree       
            out_degree2=[]   
            for component in out_degree[itteration - 1]:
                  for out_deg in component:
                        out_degree2.append(out_deg)

            #for weighted degree       
            weighted_degree2=[]   
            for component in weightd_degree[itteration - 1]:
                 for weight_deg in component:
                        weighted_degree2.append(weight_deg)            

            df = pd.DataFrame()

            df.insert(loc=0,
                      column='patients_id',
                      value=self.vertices)

            
            df.insert(loc=1,
                      column='community_id',
                      value=id)

            df.insert(loc=2,
                      column='in_degree',
                      value=weightt)
            df.insert(loc=3,
                        column='out_degree',
                        value=out_degree2)
            


            df.insert(loc=4,
                        column='weighted_degree',
                       value=weighted_degree2)
            
            

            
            return(df)
        

    def database(self):
        
        #self.interactions2=self.sql.interaction()
        
        self.allPatients=self.sql.patients()
        #self.interactions2['counter']=1

    
    def create_table_for_patients3(self,iteration2=-1):
        itteration2=iteration2
        if itteration2==-1:
            itteration=len(self.pi6)
            print(itteration)
        if itteration2 !=-1:
            itteration=itteration2
  
        itteration1=self.pi6[itteration-1]
        weightI = []
        out_degreeI = list()
        weightd_degreeI=[]
        for i in range(len(itteration1)):
            component = itteration1[i]

            subgraph = self.graph2.induced_subgraph(component)
            total_degree = self.graph2.strength(component, mode='all', loops=True, weights="weight")
            weightd_degreeI.append(total_degree)
            lst = list(range(len(component)))
            in_degree = subgraph.strength(lst, mode='all', loops=True, weights="weight")
            weightI.append(in_degree)
            lst2=[]
            for item1, item2 in zip(total_degree,in_degree ):
                item = item1 - item2
                lst2.append(item)
            out_degreeI.append(lst2) 



        if itteration2==-1:
            itteration=len(self.pi6)
            #for community id
            
            self.vertices = []
            idd = 1
            id = []
            age=[]
            gender=[]
            primary=[]
            fsa=[]
            n_encounters=[]
            n_interactions=[]
            first_admit_dt=[]
            last_admit_dt=[]
            covid_positive=[]
            
            
            
            
            
            for component in self.pi6[itteration - 1]:
                for vertex in component:
                    v= self.map4.get(vertex)
                    self.vertices.append(v)
                    id.append(idd)
                idd = idd + 1
                    
                
                   
                   #f b=label.name[index]


            #for weight
            weightt=[]
            for component in weightI:
                for degree in component:
                    weightt.append(degree)



            #for out degree       
            out_degree2=[]   
            for component in out_degreeI:
                  for out_deg in component:
                        out_degree2.append(out_deg)

           #for weighted degree       
            weighted_degree2=[]   
            for component in weightd_degreeI:
                  for weight_deg in component:
                        weighted_degree2.append(weight_deg)             





            df = pd.DataFrame()

            df.insert(loc=0,
                      column='patients_id',
                      value=self.vertices)

            
                   
            
            df.insert(loc=1,
                      column='community_id',
                      value=id)

            df.insert(loc=2,
                      column='in_degree',
                      value=weightt)
            df.insert(loc=3,
                        column='out_degree',
                        value=out_degree2)
            


            df.insert(loc=4,
                        column='weighted_degree',
                       value=weighted_degree2)
            
            

     
            return(df)

        if itteration2!=-1:
            #for id
            # idd = 1
            # id = []
            # for component in self.pi6[itteration - 1]:
                # for vertex in component:
                    # id.append(idd)
                # idd = idd + 1
                
            self.vertices = []
            idd = 1
            id = []
            age=[]
            gender=[]
            primary=[]
            fsa=[]
            n_encounters=[]
            n_interactions=[]
            first_admit_dt=[]
            last_admit_dt=[]
            covid_positive=[]
            
            
            
            
            
            for component in self.pi6[itteration - 1]:
                for vertex in component:
                    
                    self.vertices.append(self.graph2.vs[vertex]["name"])
                    '''index = np.where(self.patients.patient_id== v)[0]'''
                    id.append(idd)
                idd = idd + 1 

             

            #for weight
            weightt=[]
            for component in weightI:
                for degree in component:
                    weightt.append(degree)
            df = pd.DataFrame()                   
                    
                    

            #for out degree       
            out_degree2=[]   
            for component in out_degreeI:
                  for out_deg in component:
                        out_degree2.append(out_deg)

            #for weighted degree       
            weighted_degree2=[]   
            for component in weightd_degreeI:
                 for weight_deg in component:
                        weighted_degree2.append(weight_deg)            

                        
                        

            df = pd.DataFrame()

            df.insert(loc=0,
                      column='patients_id',
                      value=self.vertices)

            
            df.insert(loc=1,
                      column='community_id',
                      value=id)

            df.insert(loc=2,
                      column='in_degree',
                      value=weightt)
            df.insert(loc=3,
                        column='out_degree',
                        value=out_degree2)
            


            df.insert(loc=4,
                        column='weighted_degree',
                       value=weighted_degree2)
          
            return(df)
    

In [3]:
class PatientCommunityDetectionAndAnalysis:
    def __init__(self, cohort_id=-1, patient_id_df='', patient_id_query='', interactions='', data='HF'):
        self.community_detector = communityDetection(cohort_id, patient_id_df, patient_id_query, interactions)
        self.sql = sql()
        self.HF = True
        self.cohort = cohort_id


        if patient_id_df:
            patient_ids_df = pd.read_csv(patient_id_df)
            patient_ids = patient_ids_df['patient_id'].astype(str).tolist()
            interactions = find_cohort_new_interactions(cohort_id, patient_ids=patient_ids)
        else:
            interactions = find_cohort_new_interactions(cohort_id)

            

        self.Hf = data != 'HF'

        print(f"Number of unique patients in interactions: {len(interactions.patient_id.unique())}")

        if len(interactions.patient_id.unique()) >= 15900:
            print("We select 15,000 sample patients for the rest of the analysis")
            df = pd.DataFrame(interactions.patient_id.unique(), columns=['patient_id'])
            df = df.sample(n=15492)
            interactions = interactions[interactions['patient_id'].isin(df['patient_id'])]
            result = interactions
        else:
            result = interactions

        print("df")
        print(result)
        c = communityDetection(cohort_id=-1, patient_id_df='', patient_id_query='', interactions=result)
        c.create_graph_for_patients2()
        a, b = c.community_detection_on_patinets()
        self.df1 = c.create_table_for_patients3(iteration2=1)
        self.df2 = c.create_table_for_patients3(iteration2=2)
        
        # Assign the populated communityDetection instance if needed
        self.community_detector = c
        
        
    def interactionsAndPatientsOFeachSvc_HF( self,community_df):
        patient_string = '\',\''.join(community_df["patients_id"])
        patient_string = '\'' + patient_string + '\''

        query= """SELECT service_class_id,count(distinct patient_id ) as patients
          FROM HF.interactions 
          WHERE patient_id IN (
            """ + patient_string + """)"""+""" and service_class_id!=-1
            group by service_class_id
            order by patients DESC """

        return  pd.read_sql(query,engine)

    def creat_patient_svc_interaction_forCommunities(self,iteration=1):
        
        if iteration==1:
            df=self.df1
        if iteration==2:
            df=self.df2
        if self.HF==True:    
            take_cohort_svcid_label=self.take_cohort_svcid_labell_HF()
        else:
            take_cohort_svcid_label=self.take_cohort_svcid_labell_Core(self.cohort)
            
        i=0
        for community in df.community_id.unique():
            index=np.where(df.community_id==community)
            community_df=df.iloc[index]
            if self.HF==True:
                interactionsAndPatientsOFeachSvc_df= self.interactionsAndPatientsOFeachSvc_HF(community_df)
            else:
                interactionsAndPatientsOFeachSvc_df= self.interactionsAndPatientsOFeachSvc_Core(community_df)
                
                
            interactionsAndPatientsOFeachSvcMergeWithAllSvc= pd.merge(take_cohort_svcid_label, interactionsAndPatientsOFeachSvc_df, on='service_class_id', how='left')
            new_col_name = str(community)+ '  #patients'
            new_col_name2 = str(community)+ '  #interactions' 
            col_dict = {'patients':new_col_name }
            interactionsAndPatientsOFeachSvcMergeWithAllSvc=interactionsAndPatientsOFeachSvcMergeWithAllSvc.rename(columns=col_dict)
            interactionsAndPatientsOFeachSvcMergeWithAllSvc=interactionsAndPatientsOFeachSvcMergeWithAllSvc.fillna(0)
            #calculate the cohort%
            summury=interactionsAndPatientsOFeachSvcMergeWithAllSvc
            
             #summury['interaction/p'+ str(community)] = summury.apply(lambda row: safe_division(row.iloc[2], row.iloc[1]), axis=1)
            summury['community%'+ str(community)] =  summury.apply(lambda row: self.safe_division2(row.iloc[1], len(community_df)), axis=1)

            if(i!=0):
                df_copy=pd.merge( df_copy,  summury, on='service_class_id', how='left')

            if(i==0):  
                df_copy =  summury.copy(deep=True)
            i=i+1
        df_copy = df_copy.fillna(0)     
        return  df_copy
    

 

In [4]:
from py2neo import cypher, Graph, NodeMatcher, RelationshipMatcher, Subgraph, Walkable, Path, Node
import pandas as pd
import igraph
import numpy as np
from dfply import *
from IPython.display import display
from sqlalchemy import create_engine, event
from sqlalchemy.engine.url import URL
import datetime
from treelib import Node, Tree
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Trusted Connection to Named Instance
conn_str = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=KVHSQLPC56;DATABASE=AHDA;Trusted_Connection=yes;'
# Create a URL for the sqlalchemy engine to use
conn_url = URL.create("mssql+pyodbc", query={"odbc_connect": conn_str})
# Create the connection
engine = create_engine(conn_url)

def return_interaction_hf(df):
    patient_string = '\',\''.join(df["patient_id"])
    patient_string = '\'' + patient_string + '\''
    query =  """WITH FirstFall AS (""" + patient_string + """)
SELECT 
       i.patient_id, i.service_class_id, i.start_datetime, i.end_datetime
FROM 
       FirstFall
INNER JOIN 
       HF.interactions i ON i.patient_id = FirstFall.patient_id
WHERE 
       i.start_datetime < first_fall
AND        
        i.service_class_id NOT IN (80,150,152,68)
ORDER BY i.patient_id, i.start_datetime"""
    result=pd.read_sql(query,engine)
    return result

def find_cohort_new_interactions(cohort_id, patient_ids=None):
    if patient_ids is not None:
        # Construct a proper CTE with SELECT statement and column alias
        #patient_ids_str = "', '".join([str(pid) for pid in patient_ids])
        
        query =  """WITH FirstFall AS (
       SELECT 
              Distinct patient_id
              ,min([admit_datetime]) as first_fall
              ,min(diagnosis_age) as first_fall_age
       FROM 
              Core.acute_care_diagnoses acd
       WHERE 
              diagnosis_type_code NOT IN ('3', '4', '0')
              AND SUBSTRING(icd10_code, 1, 3) IN ('W06', 'W07', 'W08', 'W13', 'W14', 'W15', 'W16', 'W17', 
                                      'X80', 'Y01', 'Y30', 'W00', 'W01', 'W03', 'W04', 'W18', 
                                      'W10', 'W05')
              AND diagnosis_age >= 65
       GROUP BY 
              patient_id
)
SELECT 
       i.patient_id, i.service_class_id, i.start_datetime, i.end_datetime
FROM 
       FirstFall
INNER JOIN 
       HF.interactions i ON i.patient_id = FirstFall.patient_id
WHERE 
       i.start_datetime < first_fall
AND        
        i.service_class_id NOT IN (80,150,152,68)
ORDER BY i.patient_id, i.start_datetime"""

    elif cohort_id:
        query = """  
        SELECT  patient_id, service_class_id  
        FROM HF.interactions 
        WHERE patient_id IN (
            SELECT patient_id 
            FROM HF.patient_cohort_list 
            WHERE cohort_Id = {}
        )""".format(cohort_id)
    else:
        query = """  
            WITH FirstFall AS (
       SELECT 
              Distinct patient_id
              ,min([admit_datetime]) as first_fall
              ,min(diagnosis_age) as first_fall_age
       FROM 
              Core.acute_care_diagnoses acd
       WHERE 
              diagnosis_type_code NOT IN ('3', '4', '0')
              AND SUBSTRING(icd10_code, 1, 3) IN ('W06', 'W07', 'W08', 'W13', 'W14', 'W15', 'W16', 'W17', 
                                      'X80', 'Y01', 'Y30', 'W00', 'W01', 'W03', 'W04', 'W18', 
                                      'W10', 'W05')
              AND diagnosis_age >= 65
       GROUP BY 
              patient_id
)
SELECT 
       i.patient_id, i.service_class_id, i.start_datetime, i.end_datetime
FROM 
       FirstFall
INNER JOIN 
       HF.interactions i ON i.patient_id = FirstFall.patient_id
WHERE 
       i.start_datetime < first_fall
AND        
        i.service_class_id NOT IN (80,150,152,68)
ORDER BY i.patient_id, i.start_datetime 
        """  # No filtering by cohort_id

    # Execute the query using the engine and return the result
    result = pd.read_sql(query, engine)
    print(result)
    return result



In [5]:
# Create an instance of PatientCommunityDetectionAndAnalysis
a = PatientCommunityDetectionAndAnalysis(
    cohort_id='',
    #patient_id_df='',
    patient_id_df='patient_ids_falls.csv',
    patient_id_query='',  
    interactions='',     
    data='HF'         
)

# Access the community detection object
community_detector = a.community_detector

# Create the patient graph
patient_community_df=community_detector.create_table_for_patients3(iteration2=1)



       patient_id  service_class_id      start_datetime        end_datetime
0        10001903               154 2016-06-07 12:43:00 2016-06-07 23:59:00
1        10001903               151 2016-09-20 12:56:00 2016-09-20 23:59:00
2        10001903                82 2016-11-24 08:00:00 2016-12-07 15:14:00
3        10001903                59 2016-11-29 09:22:00 2017-04-07 10:20:00
4        10001903               154 2016-12-16 11:13:00 2016-12-16 23:59:00
...           ...               ...                 ...                 ...
144085   CCCA0484               146 2019-07-19 06:38:00 2019-07-25 14:08:00
144086   CCCA0484               138 2019-07-27 15:39:00 2019-10-14 13:53:00
144087   CCCA0484               154 2019-07-30 14:17:00 2019-07-30 23:59:00
144088   CCCA0484               154 2019-07-31 15:20:00 2019-07-31 23:59:00
144089   CCCA0484               154 2019-08-08 13:52:00 2019-08-08 23:59:00

[144090 rows x 4 columns]
Number of unique patients in interactions: 13388
df
       pa

In [6]:
print(patient_community_df.head())
patient_community_df.to_csv('patient_projection_afterfalls_without(80,150,152,68).csv', index=False)

  patients_id  community_id  in_degree  out_degree  weighted_degree
0   1005473B4             1    14235.0     12203.0          26438.0
1    10255893             1    12080.0     10595.0          22675.0
2   104568594             1    12170.0     11394.0          23564.0
3   1060696C4             1    15148.0     14877.0          30025.0
4    10CA0004             1     1060.0       527.0           1587.0


In [ ]:
df=a.creat_patient_svc_interaction_forCommunities(iteration=1)
df.to_csv('patient_svc_interaction_HFtable_Hannie_AfterFall_Without(80,150,152,68).csv', index=False)

In [ ]:
communities_df = a.df1  # Use df1 from the first iteration for the community data

# Count unique patients in each community
community_patient_counts = communities_df.groupby('community_id')['patients_id'].nunique().reset_index()
community_patient_counts.columns = ['community_id', 'unique_patient_count']

# Display the result
print(community_patient_counts)